### Modeling of manually-derived strains
In this section, we are going to evaluate some popular manual derived changes through model simulations. Those are the downregulation of squalene synthase (<i>ERG9</i>), squalene epoxidase (<i>ERG1</i>), and sterol C-5 desaturase (<i>ERG3</i>) the upregulation of 3-hydroxy-3-methylglutaryl-coenzyme A reductase (<i>HMG1</i>) and the deletion of aldehyde dehydrogenase (<i>ALD6</i>).

Loading the model

In [61]:
from cobra.io import read_sbml_model, write_sbml_model
model = read_sbml_model('/Users/nikow/github-classroom/27410/group-assingment-team-4/yeast-GEM-8.7.0/model/yeast-GEM3.xml') # loading
from cobra import Reaction, Metabolite, Gene
import pandas as pd

In [62]:
model.optimize

<bound method Model.optimize of <Model yeastGEM_v8__46__7__46__0 at 0x18e73958950>>

We will locate the reactions in the model

In [63]:
model.reactions.query('squalene synthase', 'name')

[<Reaction r_1012 at 0x18e759b8650>]

In [64]:
model.reactions.r_1012

Reaction identifier,r_1012
Name,squalene synthase
Memory address,0x18e759b8650
Stoichiometry,2.0 s_0190 + s_0794 + s_1212 --> 2.0 s_0633 + s_1207 + s_1447 2.0 farnesyl diphosphate + H+ + NADPH --> 2.0 diphosphate + NADP(+) + squalene
GPR,YHR190W
Lower bound,0.0
Upper bound,1000.0


In [65]:
model.reactions.query('squalene epoxidase', 'name')

[<Reaction r_1010 at 0x18e759c6310>, <Reaction r_1011 at 0x18e759387d0>]

In [66]:
model.reactions.r_1010

Reaction identifier,r_1010
Name,squalene epoxidase (NAD)
Memory address,0x18e759c6310
Stoichiometry,"s_0795 + s_1204 + s_1276 + s_1448 --> s_0038 + s_0804 + s_1199 H+ + NADH + oxygen + squalene --> (S)-2,3-epoxysqualene + H2O + NAD"
GPR,YGR175C
Lower bound,0.0
Upper bound,1000.0


In [67]:
model.reactions.query('sterol desaturase', 'name')

[<Reaction r_0233 at 0x18e75712110>, <Reaction r_0242 at 0x18e757174d0>]

In [68]:
model.reactions.r_0242

Reaction identifier,r_0242
Name,C-5 sterol desaturase
Memory address,0x18e757174d0
Stoichiometry,"s_0657 + s_0794 + s_1212 + s_1275 --> s_0664 + 2.0 s_0803 + s_1207 episterol + H+ + NADPH + oxygen --> ergosta-5,7,24(28)-trien-3beta-ol + 2.0 H2O + NADP(+)"
GPR,YLR056W
Lower bound,0.0
Upper bound,1000.0


In [69]:
model.reactions.query('hydroxymethylglutaryl CoA reductase', 'name')

[<Reaction r_0558 at 0x18e75836590>]

In [70]:
model.reactions.r_0558

Reaction identifier,r_0558
Name,hydroxymethylglutaryl CoA reductase
Memory address,0x18e75836590
Stoichiometry,s_0218 + 2.0 s_0794 + 2.0 s_1212 --> s_0028 + s_0529 + 2.0 s_1207 3-hydroxy-3-methylglutaryl-CoA + 2.0 H+ + 2.0 NADPH --> (R)-mevalonate + coenzyme A + 2.0 NADP(+)
GPR,YLR450W or YML075C
Lower bound,0.0
Upper bound,1000.0


<i>S. cerevisiae</i> posseses two genes encoding 3-hydroxy-3-methylglutaryl-coenzyme A reductase enzymes, <i>HMG1</i> and <i>HMG2</i>. As the isoenzyme HMG1 appears to be the more stable version of the two, it is the one usually picked for upregulation.

In [71]:
model.reactions.query('YPL061W' , 'gene_name_reaction_rule')

[<Reaction r_0173 at 0x18e756b34d0>,
 <Reaction r_0177 at 0x18e756be310>,
 <Reaction r_4718 at 0x18e76bf5fd0>,
 <Reaction r_4719 at 0x18e76ba7f10>]

In [72]:
model.reactions.r_0173

Reaction identifier,r_0173
Name,"aldehyde dehydrogenase (acetaldehyde, NADP)"
Memory address,0x18e756b34d0
Stoichiometry,s_0359 + s_0803 + s_1207 --> s_0362 + 2.0 s_0794 + s_1212 acetaldehyde + H2O + NADP(+) --> acetate + 2.0 H+ + NADPH
GPR,YPL061W
Lower bound,0.0
Upper bound,1000.0


In [73]:
model.reactions.r_0177

Reaction identifier,r_0177
Name,"aldehyde dehydrogenase (indole-3-acetaldehyde, NADP)"
Memory address,0x18e756be310
Stoichiometry,s_0803 + s_0850 + s_1207 --> 2.0 s_0794 + s_0853 + s_1212 H2O + indol-3-ylacetaldehyde + NADP(+) --> 2.0 H+ + indole-3-acetate + NADPH
GPR,YPL061W
Lower bound,0.0
Upper bound,1000.0


In [74]:
model.reactions.r_4718

Reaction identifier,r_4718
Name,"isobutyraldehyde dehydrogenase, c"
Memory address,0x18e76bf5fd0
Stoichiometry,s_0803 + s_0937 + s_1198 <=> 2.0 s_0794 + s_1203 + s_4285 H2O + isobutyraldehyde + NAD <=> 2.0 H+ + NADH + isobutyrate
GPR,YOR374W or YPL061W
Lower bound,-1000.0
Upper bound,1000.0


In [75]:
model.reactions.r_4719

Reaction identifier,r_4719
Name,"2-methylbutanal dehydrogenase, c"
Memory address,0x18e76ba7f10
Stoichiometry,s_0166 + s_0803 + s_1198 <=> 2.0 s_0794 + s_1203 + s_4282 2-methylbutanal + H2O + NAD <=> 2.0 H+ + NADH + 2-methylbutyrate
GPR,YOR374W or YPL061W
Lower bound,-1000.0
Upper bound,1000.0


In [76]:
model.reactions.query('YOR374W' , 'gene_name_reaction_rule')

[<Reaction r_0174 at 0x18e756bef50>,
 <Reaction r_0175 at 0x18e756be110>,
 <Reaction r_0176 at 0x18e756b3310>,
 <Reaction r_0178 at 0x18e756c6c10>,
 <Reaction r_0201 at 0x18e756293d0>,
 <Reaction r_2116 at 0x18e75c853d0>,
 <Reaction r_4678 at 0x18e76bc6990>,
 <Reaction r_4718 at 0x18e76bf5fd0>,
 <Reaction r_4719 at 0x18e76ba7f10>]

It appears that ALD6 catalyzes 4 reactions in total, for two of them being the only enzyme doing so. For the later two, another aldehyde dehydrogenase, ALD4, has the abillity to catalyze the same reactions. As a conclusion, to demosntrate the deletion of <i>ALD6</i>, we will delete reactions r_0173 and r_0177, as they are the main reactions our enzyme is involved into.

The growth rate, productivity and yield for the base strain are displayed, for comparison with the following modifications.

In [77]:
with model:
    growth_rate = model.optimize().objective_value
    print('Growth rate:', growth_rate, 'gDW h^-1')

Growth rate: 0.7486631103217283 gDW h^-1


In [78]:
with model:
    model.objective = model.reactions.BFD

    beta_farnesene_max_production = model.optimize().objective_value
    print("Maximum beta-farnesene productivity = ",beta_farnesene_max_production , "mmol/gDW*h")

    glucose_uptake_flux = model.reactions.r_1714.flux #Glucose exchange reaction
    max_yield = beta_farnesene_max_production / (-1*glucose_uptake_flux)
    print("Maximum theoretical yield is = ", max_yield, "mmol/gDW*h")

Maximum beta-farnesene productivity =  5.138027093642488 mmol/gDW*h
Maximum theoretical yield is =  0.2569013546821244 mmol/gDW*h


For the next step, we will simulate the modifications in our model.

<i>ALD6</i> deletion

In [79]:
model.reactions.r_0173

Reaction identifier,r_0173
Name,"aldehyde dehydrogenase (acetaldehyde, NADP)"
Memory address,0x18e756b34d0
Stoichiometry,s_0359 + s_0803 + s_1207 --> s_0362 + 2.0 s_0794 + s_1212 acetaldehyde + H2O + NADP(+) --> acetate + 2.0 H+ + NADPH
GPR,YPL061W
Lower bound,0.0
Upper bound,1000.0


In [80]:
model.reactions.r_0177

Reaction identifier,r_0177
Name,"aldehyde dehydrogenase (indole-3-acetaldehyde, NADP)"
Memory address,0x18e756be310
Stoichiometry,s_0803 + s_0850 + s_1207 --> 2.0 s_0794 + s_0853 + s_1212 H2O + indol-3-ylacetaldehyde + NADP(+) --> 2.0 H+ + indole-3-acetate + NADPH
GPR,YPL061W
Lower bound,0.0
Upper bound,1000.0


In [81]:
with model:
    model.reactions.r_0173.knock_out()
    model.reactions.r_0177.knock_out()
    
    growth_rate = model.optimize().objective_value
    print('Growth rate after deletion:', growth_rate, 'gDW h^-1')
    
    

Growth rate after deletion: 0.7486631103217309 gDW h^-1


In [82]:
with model:
    model.objective = model.reactions.BFD

    beta_farnesene_max_production = model.optimize().objective_value
    print("Maximum beta-farnesene productivity = ",beta_farnesene_max_production , "mmol/gDW*h")

    glucose_uptake_flux = model.reactions.r_1714.flux #Glucose exchange reaction
    max_yield = beta_farnesene_max_production / (-1*glucose_uptake_flux)
    print("Maximum theoretical yield is = ", max_yield, "mmol/gDW*h")


Maximum beta-farnesene productivity =  5.13802709364245 mmol/gDW*h
Maximum theoretical yield is =  0.2569013546821225 mmol/gDW*h


<i>HMG1</i> upregulation

We are going to simulate the upregulation of <i>HMG1</i> by increasing the lower bound of the HMGR reaction in our model to 0.1 from 0.0.


In [83]:
with model:
    model.reactions.r_0558.lower_bound = 0.1
    
    growth_rate = model.optimize().objective_value
    print('Growth rate after upregulation:', growth_rate, 'gDW h^-1')
    
    

Growth rate after upregulation: 0.7486631103217283 gDW h^-1


In [84]:
with model:
    model.objective = model.reactions.BFD

    beta_farnesene_max_production = model.optimize().objective_value
    print("Maximum beta-farnesene productivity = ",beta_farnesene_max_production , "mmol/gDW*h")

    glucose_uptake_flux = model.reactions.r_1714.flux #Glucose exchange reaction
    max_yield = beta_farnesene_max_production / (-1*glucose_uptake_flux)
    print("Maximum theoretical yield is = ", max_yield, "mmol/gDW*h")


Maximum beta-farnesene productivity =  5.138027093642464 mmol/gDW*h
Maximum theoretical yield is =  0.25690135468212316 mmol/gDW*h


<i>ERG9</i> downregulation

Next, we are going to downregulate the expression of <i>ERG9</i> in our model, by decreasing the upper bound of the reaction. Remember that deletion of that gene is lethal, so we have to be careful with how much we are decrease the upper bound. After consideration, we choose the value of 0.1.

In [85]:
model.reactions.r_1012

Reaction identifier,r_1012
Name,squalene synthase
Memory address,0x18e759b8650
Stoichiometry,2.0 s_0190 + s_0794 + s_1212 --> 2.0 s_0633 + s_1207 + s_1447 2.0 farnesyl diphosphate + H+ + NADPH --> 2.0 diphosphate + NADP(+) + squalene
GPR,YHR190W
Lower bound,0.0
Upper bound,1000.0


In [86]:
with model:
    model.reactions.r_1012.upper_bound = 0.1
    
    growth_rate = model.optimize().objective_value
    print('Growth rate after downregulation:', growth_rate, 'gDW h^-1')
    
    model.objective = model.reactions.BFD

    beta_farnesene_max_production = model.optimize().objective_value
    print("Maximum beta-farnesene productivity = ",beta_farnesene_max_production , "mmol/gDW*h")

    glucose_uptake_flux = model.reactions.r_1714.flux #Glucose exchange reaction
    max_yield = beta_farnesene_max_production / (-1*glucose_uptake_flux)
    print("Maximum theoretical yield is = ", max_yield, "mmol/gDW*h")

Growth rate after downregulation: 0.7486631103217283 gDW h^-1
Maximum beta-farnesene productivity =  5.138027093642445 mmol/gDW*h
Maximum theoretical yield is =  0.2569013546821223 mmol/gDW*h


In the same way as <i>ERG9</i> we are going to model the downregulation of <i>ERG1</i> and <i>ERG3</i>.

Downregulation of <i>ERG1</i>

In [87]:
with model:
    model.reactions.r_1010.upper_bound = 0.1
    
    growth_rate = model.optimize().objective_value
    print('Growth rate after downregulation:', growth_rate, 'gDW h^-1')
    
    model.objective = model.reactions.BFD

    beta_farnesene_max_production = model.optimize().objective_value
    print("Maximum beta-farnesene productivity = ",beta_farnesene_max_production , "mmol/gDW*h")

    glucose_uptake_flux = model.reactions.r_1714.flux #Glucose exchange reaction
    max_yield = beta_farnesene_max_production / (-1*glucose_uptake_flux)
    print("Maximum theoretical yield is = ", max_yield, "mmol/gDW*h")

Growth rate after downregulation: 0.7486631103217282 gDW h^-1
Maximum beta-farnesene productivity =  5.1380270936424495 mmol/gDW*h
Maximum theoretical yield is =  0.2569013546821225 mmol/gDW*h


Downregulation of <i>ERG3</i>

In [88]:
with model:
    model.reactions.r_0242.upper_bound = 0.1
    
    growth_rate = model.optimize().objective_value
    print('Growth rate after downregulation:', growth_rate, 'gDW h^-1')
    
    model.objective = model.reactions.BFD

    beta_farnesene_max_production = model.optimize().objective_value
    print("Maximum beta-farnesene productivity = ",beta_farnesene_max_production , "mmol/gDW*h")

    glucose_uptake_flux = model.reactions.r_1714.flux #Glucose exchange reaction
    max_yield = beta_farnesene_max_production / (-1*glucose_uptake_flux)
    print("Maximum theoretical yield is = ", max_yield, "mmol/gDW*h")

Growth rate after downregulation: 0.7486631103217288 gDW h^-1
Maximum beta-farnesene productivity =  5.138027093642449 mmol/gDW*h
Maximum theoretical yield is =  0.25690135468212244 mmol/gDW*h


### Conclusions
We simulated some frequently used in literature manually derived metabolic modifications in <i>S. cerevisiae</i> in order to increase terpenoid (and beta-farnesene in our case) production. 
Our analysis showed that none of the modifications implemented had any substantial results in both growth rate and production in silico. This might be explained due to the absence of regulatory and kinetic parameters in our model, which could simulate the metabolic processes involved in β-farnesene production in a more accurate way than just using stoichiometry.


In [89]:
model.medium

{'r_1654': 1000.0,
 'r_1714': 20.0,
 'r_1810': 0.24,
 'r_1832': 1000.0,
 'r_1861': 0.0005,
 'r_1873': 0.44,
 'r_1879': 0.25,
 'r_1881': 0.49,
 'r_1883': 0.04,
 'r_1891': 0.81,
 'r_1893': 0.1,
 'r_1897': 0.27,
 'r_1899': 0.38,
 'r_1900': 0.4,
 'r_1902': 0.09,
 'r_1903': 0.23,
 'r_1904': 0.2,
 'r_1906': 0.23,
 'r_1911': 0.21,
 'r_1912': 0.14,
 'r_1913': 0.06,
 'r_1914': 0.29,
 'r_1992': 1000.0,
 'r_2005': 1000.0,
 'r_2020': 1000.0,
 'r_2049': 1000.0,
 'r_2060': 1000.0,
 'r_2100': 1000.0,
 'r_4593': 1000.0,
 'r_4594': 1000.0,
 'r_4595': 1000.0,
 'r_4596': 0.00056,
 'r_4597': 1000.0,
 'r_4600': 1000.0}